## Data Wrangling

### Introduction

This project is part of a Capstone project for Springboard Data Science Career Track. The goal of this project is to develop a machine learning model to rank and predict the likelihood that an oil company will initiate a frac job in a county within the Permian Basin in the first quarter of 2024.

In [1]:
# initial imports

import pandas as pd
import numpy as np
from urllib.request import urlopen

In [2]:
# Test initial print statement
print("CapstoneJourney begins!")

CapstoneJourney begins!


In [3]:
# there is FracFocusRegistry_i.csv files in the bucket for i in range 1-24
# there is registryupload_i.csv files in the bucket for i in range 1-3

# First list of urls
data_urls1 = []
for i in range(1, 25):
    url_frame = f"https://storage.googleapis.com/mrprime_dataset/fracfocus/FracFocusRegistry_{i}.csv"
    data_urls1.append(url_frame)

# Second list of urls
data_urls2 = []
for j in range(1, 4):
    url_frame2 = f"https://storage.googleapis.com/mrprime_dataset/fracfocus/registryupload_{j}.csv"
    data_urls2.append(url_frame2)

In [4]:
dfs = [pd.read_csv(url) for url in data_urls2]

pd.concat(dfs)

C:\Users\jonnr\AppData\Local\Temp\ipykernel_24964\2578461586.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(url) for url in data_urls2]


,pKey,JobStartDate,JobEndDate,APINumber,StateNumber,CountyNumber,OperatorName,WellName,Latitude,Longitude,...,TVD,TotalBaseWaterVolume,TotalBaseNonWaterVolume,StateName,CountyName,FFVersion,FederalWell,IndianWell,Source,DTMOD
0,448c1dab-c7fd-4e07-9d6f-e3b1cf64b708,5/1/1955 12:00:00 AM,5/1/1955 12:00:00 AM,42317372620000,42,317,Pioneer Natural Resources,Rogers 42 #5,32.283431,-101.906575,...,NaN,NaN,NaN,Texas,Martin,1,False,False,NaN,NaN
1,f66add2e-8ea8-4843-9388-24725b5d37c1,5/19/1982 12:00:00 AM,5/19/1982 12:00:00 AM,49009219470000,49,9,"Chesapeake Operating, Inc.",WILLIAM VALENTINE 1,42.972810,-105.953840,...,NaN,NaN,NaN,WYOMING,CONVERSE,1,False,False,NaN,NaN
2,95f0904c-2556-4912-9f5a-34913ba57625,2/7/1995 12:00:00 AM,2/7/1995 12:00:00 AM,49009228850000,49,9,"Chesapeake Operating, Inc.",LIZARD HEAD 1-8H RE,42.851470,-105.411510,...,NaN,NaN,NaN,WYOMING,CONVERSE,1,False,False,NaN,NaN
3,5a9513e7-b79f-4488-9f64-fa0960e595f3,6/11/1996 12:00:00 AM,6/11/1996 12:00:00 AM,42335355480000,42,335,Energen Resources Corporation,North Westbrook Unit/Well No. 3032,32.440260,-101.030130,...,NaN,NaN,NaN,Texas,Mitchell,1,False,False,NaN,NaN
4,36368c95-a097-40ea-bb9f-2b647de7b913,12/13/2001 12:00:00 AM,12/13/2001 12:00:00 AM,42395313840000,42,395,XTO Energy/ExxonMobil,Olene Reagan 3-1,31.257980,-96.374820,...,NaN,NaN,NaN,Texas,Robertson,1,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7359,b33502b6-f5fd-47ce-98b2-6a9d77781a81,5/3/2023 5:00:00 AM,5/12/2023 5:00:00 AM,42033326180000,42,33,"Surge Operating, LLC",Erlking Unit A 21-09 3AH,32.532688,-101.527801,...,7539.0,20414722.0,0.0,Texas,Borden,3,False,False,NaN,NaN
7360,1554d6f1-24a1-452c-92cf-9b3db916fea3,5/3/2023 5:00:00 AM,5/12/2023 5:00:00 AM,42033326190000,42,33,"Surge Operating, LLC",Erlking Unit B 21-09 4AH,32.532623,-101.527778,...,7500.0,20393764.0,0.0,Texas,Borden,3,False,False,NaN,NaN
7361,cae35d55-08cf-4f67-8a8c-9735509be73a,5/2/2023 2:16:00 AM,5/13/2023 7:37:00 PM,49005782270000,49,5,Anschutz Exploration Corporation,Tex Fed 4271-17-20-15 MH,43.619864,-105.407289,...,10568.0,21963648.0,0.0,Wyoming,Campbell,3,True,False,NaN,NaN
7362,a63c66c9-2129-456d-b503-a850c34e1ffd,5/1/2023 12:06:00 PM,5/13/2023 7:37:00 PM,49005782280000,49,5,Anschutz Exploration Corporation,Tex Fed 4271-17-20-16 MH,43.619812,-105.407268,...,10583.0,21994812.0,0.0,Wyoming,Campbell,3,True,False,NaN,NaN
